In [83]:
import pandas as pd
import sqlalchemy
from routes import routes
from utils import graph_utils, station_manager, visualization_utils

In [84]:
engine = sqlalchemy.create_engine('sqlite:///../data/processed/database.db')

def import_query(file_path):
    with open(file_path, 'r') as file:
        return file.read()

query = """SELECT 
    o.data,
    o.id,
    o.TU,
    o.ValorTKU,
    f.ferrovia,
    f.cliente,
    f.mercadoria,
    f.origem,
    f.destino,
    f.dist_media,
    f.intermed,
    f.len_Dijkstra,
    f.rota
FROM 
    ocorrencias o
LEFT JOIN 
    fluxos_rotas_2017 f
ON 
    o.id = f.id
WHERE 
    o.data >= '2017-01-01' and o.data <= '2024-12-01';
"""
fluxos = pd.read_sql(query, engine)
fluxos['year'] = pd.to_datetime(fluxos['data']).dt.year

In [85]:
# import stations
stations = routes.extracts('../data/processed/stations.gpkg')
remover = routes.select_criteria(stations) # filter stations (same criteria as routes)
stations = station_manager.remove_stations(stations, remover) # remover as estações deletadas; aqui tem que ser o mesmo critério de routes
G = graph_utils.generate_graph(stations)

rmp_stations = stations[stations['ferrovia'] == 'RMP']['estacao'].unique().tolist()
rmn_stations = stations[stations['ferrovia'] == 'RMN']['estacao'].unique().tolist()
rmc_stations = stations[stations['ferrovia'] == 'RMC']['estacao'].unique().tolist()

In [86]:
fluxos['RMP'] = fluxos.apply(lambda row: graph_utils.find_tu(G, row['rota'], rmp_stations, row['TU'],'RMP'), axis=1)
fluxos['RMN'] = fluxos.apply(lambda row: graph_utils.find_tu(G, row['rota'], rmn_stations, row['TU'],'RMN'), axis=1)
fluxos['RMC'] = fluxos.apply(lambda row: graph_utils.find_tu(G, row['rota'], rmc_stations, row['TU'],'RMC'), axis=1)

In [87]:
# aplicar a função bool em 
fluxos['bool_TU'] = fluxos.apply(lambda row: bool(row['RMP'] +row['RMN']+row['RMC']), axis=1)
fluxos = fluxos[fluxos['bool_TU'] == True]

In [88]:
fluxos['RMP_tku'] = fluxos.apply(lambda row: graph_utils.find_tku(G, row['rota'], rmp_stations, row['TU'],'RMP'), axis=1)
fluxos['RMN_tku'] = fluxos.apply(lambda row: graph_utils.find_tku(G, row['rota'], rmn_stations, row['TU'],'RMN'), axis=1)
fluxos['RMC_tku'] = fluxos.apply(lambda row: graph_utils.find_tku(G, row['rota'], rmc_stations, row['TU'],'RMC'), axis=1)

fluxos['soma_tku'] = fluxos['RMP_tku'] + fluxos['RMN_tku'] + fluxos['RMC_tku']

# Visualizações

In [89]:
# geral:
with pd.option_context('display.float_format', '{:,.0f}'.format):
    display(fluxos.groupby(['year']).agg({'TU':'sum', 'RMP':'sum', 'RMN':'sum', 'RMC':'sum'}).T)

year,2017,2018,2019,2020,2021,2022,2023,2024
TU,"43,886,529","48,287,020","49,962,659","52,978,048","52,955,727","62,061,617","66,430,147","64,366,210"
RMP,"43,867,310","48,287,020","49,962,659","52,978,048","52,955,455","62,061,617","66,428,264","64,362,160"
RMN,"21,603,805","25,724,907","28,616,177","29,820,293","28,886,047","33,382,917","33,863,268","33,754,583"
RMC,"19,219",0,0,0,"3,359,145","7,845,362","8,722,244","8,111,149"


In [107]:
# geral:
with pd.option_context('display.float_format', '{:,.0f}'.format):
    display(fluxos[fluxos['ferrovia']=='RMP'].groupby(['year']).agg({'TU':'sum', 'RMP':'sum', 'RMN':'sum', 'RMC':'sum'}).T)

year,2017,2018,2019,2020,2021,2022,2023,2024
TU,"4,051,489","5,268,201","5,818,185","8,097,876","7,882,248","6,258,054","6,198,135","6,538,691"
RMP,"4,051,489","5,268,201","5,818,185","8,097,876","7,882,248","6,258,054","6,198,135","6,538,691"
RMN,"1,216,472","2,059,686","2,983,963","3,625,349","4,186,589","4,324,038","4,373,746","4,260,048"
RMC,0,0,0,0,0,0,"8,044","840,139"


In [109]:
# RMP:
with pd.option_context('display.float_format', '{:,.0f}'.format):
    display(fluxos[fluxos['ferrovia']=='RMC'].groupby(['year']).agg({'TU':'sum', 'RMP_tku':'sum',
                               'RMN_tku':'sum', 'RMC_tku':'sum','soma_tku': 'sum'}).T)

year,2021,2022,2023,2024
TU,"3,359,145","7,845,362","8,712,403","7,267,037"
RMP_tku,"2,680,251,718","6,259,790,804","6,951,600,216","5,787,360,802"
RMN_tku,0,0,0,0
RMC_tku,"860,558,843","2,569,148,473","2,798,641,694","2,381,165,905"
soma_tku,"3,540,810,561","8,828,939,276","9,750,241,910","8,168,526,707"


In [110]:
# RMP:
with pd.option_context('display.float_format', '{:,.0f}'.format):
    display(fluxos.groupby(['year']).agg({'TU':'sum', 'RMP_tku':'sum',
                               'RMN_tku':'sum', 'RMC_tku':'sum','soma_tku': 'sum'}).T)

year,2017,2018,2019,2020,2021,2022,2023,2024
TU,"43,886,529","48,287,020","49,962,659","52,978,048","52,955,727","62,061,617","66,430,147","64,366,210"
RMP_tku,"26,005,013,047","29,547,072,739","31,653,751,515","33,629,763,537","34,457,908,699","41,805,809,629","43,898,486,879","42,908,513,772"
RMN_tku,"14,280,081,467","16,318,459,635","17,861,095,738","18,601,167,617","18,633,064,714","21,526,226,920","22,126,781,778","21,915,611,677"
RMC_tku,"11,232,981",0,0,0,"860,558,843","2,569,148,473","2,807,927,657","2,697,619,208"
soma_tku,"40,296,327,496","45,865,532,375","49,514,847,253","52,230,931,153","53,951,532,256","65,901,185,022","68,833,196,314","67,521,744,657"


In [108]:
# RMP:
with pd.option_context('display.float_format', '{:,.0f}'.format):
    display(fluxos.groupby(['year']).agg({'TU':'sum', 'RMP_tku':'sum',
                               'RMN_tku':'sum', 'RMC_tku':'sum','soma_tku': 'sum'}).T)

year,2017,2018,2019,2020,2021,2022,2023,2024
TU,"43,886,529","48,287,020","49,962,659","52,978,048","52,955,727","62,061,617","66,430,147","64,366,210"
RMP_tku,"26,005,013,047","29,547,072,739","31,653,751,515","33,629,763,537","34,457,908,699","41,805,809,629","43,898,486,879","42,908,513,772"
RMN_tku,"14,280,081,467","16,318,459,635","17,861,095,738","18,601,167,617","18,633,064,714","21,526,226,920","22,126,781,778","21,915,611,677"
RMC_tku,"11,232,981",0,0,0,"860,558,843","2,569,148,473","2,807,927,657","2,697,619,208"
soma_tku,"40,296,327,496","45,865,532,375","49,514,847,253","52,230,931,153","53,951,532,256","65,901,185,022","68,833,196,314","67,521,744,657"


In [93]:
# RMN:
with pd.option_context('display.float_format', '{:,.0f}'.format):
    display(fluxos[fluxos['ferrovia']=='RMN'].groupby(['year']).agg({'TU':'sum', 'RMP_tku':'sum',
                               'RMN_tku':'sum', 'RMC_tku':'sum','soma_tku': 'sum'}).T)

year,2017,2018,2019,2020,2021,2022,2023,2024
TU,"20,387,333","23,665,221","25,632,214","26,194,944","24,699,458","29,058,879","29,489,522","29,494,535"
RMP_tku,"17,921,175,011","20,735,577,112","22,415,454,245","22,874,291,650","21,407,031,058","25,166,572,844","25,496,199,214","25,460,386,609"
RMN_tku,"13,464,114,622","14,855,991,077","15,705,309,514","15,948,147,089","15,568,902,222","18,376,704,124","18,940,278,639","18,808,321,861"
RMC_tku,0,0,0,0,0,0,0,0
soma_tku,"31,385,289,633","35,591,568,189","38,120,763,759","38,822,438,739","36,975,933,280","43,543,276,968","44,436,477,852","44,268,708,470"


In [92]:
# RMC:
with pd.option_context('display.float_format', '{:,.0f}'.format):
    display(fluxos[fluxos['ferrovia']=='RMC'].groupby(['year']).agg({'TU':'sum', 'RMP_tku':'sum',
                               'RMN_tku':'sum', 'RMC_tku':'sum','soma_tku': 'sum'}).T)

year,2021,2022,2023,2024
TU,"3,359,145","7,845,362","8,712,403","7,267,037"
RMP_tku,"2,680,251,718","6,259,790,804","6,951,600,216","5,787,360,802"
RMN_tku,0,0,0,0
RMC_tku,"860,558,843","2,569,148,473","2,798,641,694","2,381,165,905"
soma_tku,"3,540,810,561","8,828,939,276","9,750,241,910","8,168,526,707"


In [100]:
fluxos['TKU_total'] = fluxos['TU'] * fluxos['dist_media']

In [111]:
# Por mercadoria: recorte RMP
# unstack the year index from fluxos.groupby(['mercadoria','destino','year']).agg({'RMP':'sum'}):
df = fluxos[(fluxos['year']>2021) & (fluxos['ferrovia']=='RMC')].groupby(['mercadoria','year']).agg({'RMC_tku':'sum', 'TKU_total':'sum'}).unstack()
df = df.fillna(0)
df = df.loc[(df!=0).any(axis=1)] # drop rows with all zeros
with pd.option_context('display.float_format', '{:,.0f}'.format):
    display(df)


RMC_tku                              \
year                                     2022          2023          2024   
mercadoria                                                                  
Adubo Orgânico a Granel            42,236,741    53,765,069    50,381,794   
Alumínio                                    0             0     4,004,272   
Açúcar                             56,046,438    99,240,170   111,127,994   
Cloreto de Potássio                48,660,519   103,157,929    96,035,737   
Contêiner Cheio de 20 Pés                   0       441,841        95,513   
Contêiner Cheio de 40 Pés                   0     4,889,412    12,452,470   
Contêiner Vazio de 20 Pés                   0        69,621        49,579   
Contêiner Vazio de 40 Pés                   0     1,173,138     3,381,612   
Farelo de Soja                    298,242,234   351,357,617   326,610,502   
Fosfato                            12,215,044     6,910,446    11,157,088   
Grãos - Milho                   1,008,303,455   970,111,799   534,702,310   
Outros - adubos e Fertilizantes             0             0     2,105,773   
Pallets de Madeira                          0        88,951             0   
Soja                            1,080,984,021 1,183,976,996 1,212,809,761   
Ureia                              22,460,021    23,458,703    16,251,500   

                                    TKU_total                              
year                                     2022          2023          2024  
mercadoria                                                                 
Adubo Orgânico a Granel           131,876,812   167,365,529   155,592,595  
Alumínio                                    0             0     7,552,214  
Açúcar                            576,706,627 1,021,579,775 1,144,875,421  
Cloreto de Potássio               151,726,625   321,804,141   294,690,944  
Contêiner Cheio de 20 Pés                   0       927,361       200,469  
Contêiner Cheio de 40 Pés                   0    10,262,185    26,135,977  
Contêiner Vazio de 20 Pés                   0       151,342       104,060  
Contêiner Vazio de 40 Pés                   0     2,462,251     6,692,931  
Farelo de Soja                    927,625,206 1,092,485,782 1,051,794,477  
Fosfato                            38,114,885    21,604,912    34,093,928  
Grãos - Milho                   3,391,596,196 3,241,700,586 1,647,885,300  
Outros - adubos e Fertilizantes             0             0     6,498,333  
Pallets de Madeira                          0       186,696             0  
Soja                            3,731,033,439 4,002,043,220 3,905,308,367  
Ureia                              70,112,276    72,997,876    50,280,793

In [105]:
# Por mercadoria: recorte RMP
# unstack the year index from fluxos.groupby(['mercadoria','destino','year']).agg({'RMP':'sum'}):
df = fluxos[fluxos['year']>2021].groupby(['mercadoria','destino','year']).agg({'RMP_tku':'sum', 'TKU_total':'sum'}).unstack()
df = df.fillna(0)
df = df.loc[(df!=0).any(axis=1)] # drop rows with all zeros
with pd.option_context('display.float_format', '{:,.0f}'.format):
    display(df.head(50))


RMP_tku                \
year                                                      2022          2023   
mercadoria                               destino                               
Adubo Fert em Geral a Granel  - Perigoso TRO           423,452             0   
Adubo Orgânico a Granel                  PRV        86,421,022   112,621,566   
                                         TRO     1,131,291,082 1,461,208,629   
                                         ZVF                 0             0   
Alumínio                                 ZAL                 0             0   
Açúcar                                   ICB                 0             0   
                                         ICZ       135,303,327   236,965,698   
                                         ISN     2,928,058,605 2,942,840,099   
                                         IUF       972,978,085 1,224,805,147   
Bauxita                                  ZAL       231,970,187   237,727,563   
Celulose                                 IBA     2,334,577,570 2,024,446,178   
                                         ISN     1,063,265,758 1,245,732,451   
                                         ZPD                 0             0   
Cimento acondicionado                    ZZZ             3,746             0   
Cloreto de Potássio                      EBJ         6,041,572             0   
                                         IFA       100,069,607    99,764,617   
                                         PRV        99,564,779   211,072,481   
                                         TRO       829,554,810   663,299,051   
                                         ZVF         2,896,900    29,191,737   
Contêiner Cheio de 20 Pés                ICB        17,877,288    87,322,015   
                                         PAS                 0        16,756   
                                         TRO         1,047,876     4,424,491   
                                         ZBV            62,699        41,522   
Contêiner Cheio de 20 Pés - Perigoso     ZZZ             2,399             0   
Contêiner Cheio de 40 Pés                FBA                 0             0   
                                         IBA             8,667             0   
                                         ICB       599,633,213   536,571,864   
                                         ICZ                 0       142,617   
                                         IFA                 0        81,707   
                                         PAS                 0     1,943,677   
                                         TRO       222,555,927   209,872,610   
                                         ZBL            17,094    68,624,801   
                                         ZBV       367,319,128   362,888,116   
                                         ZOP            37,294             0   
                                         ZTO        41,070,566     3,646,177   
                                         ZZZ                 0         5,998   
Contêiner Vazio de 20 Pés                ICB            89,870       189,606   
                                         PAS                 0        70,215   
                                         TRO         2,296,947     9,242,202   
                                         ZBL            45,584             0   
Contêiner Vazio de 40 Pés                ICB           839,284     1,387,886   
                                         ICZ             4,329             0   
                                         IFA                 0             0   
                                         PAS                 0       915,188   
                                         TRO        99,828,630    93,344,620   
                                         ZBL           779,993     1,288,381   
                                         ZBV            68,236       568,189   
                                         ZTO           